In [49]:
from sqlalchemy import create_engine
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
import panel as pn
import plotly.graph_objects as go
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator
import os


In [2]:
# Usar o Plotly como visualização
pn.extension("plotly")

In [32]:
# Carregando o Vader para Sentiment Analysis
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Claudenir\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [33]:
# Carregando os envs
load_dotenv()

True

In [22]:
# Conexão com o Banco de Dados
usuario = os.getenv("DB_USER")
senha = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
nome_do_banco = os.getenv("DB_NAME")

engine = create_engine(
    f"postgresql+psycopg2://{usuario}:{senha}@{host}:{port}/{nome_do_banco}"
)

In [76]:
cliente = pd.read_sql("cliente", con=engine)
feedback = cliente.feedback.dropna()
print(feedback.head())


0       O serviço foi razoável, mas poderia ser melhor
1            Excelente, o serviço foi rápido e eficaz.
3     Não tenho grandes queixas, mas também não há ...
5    Fiquei impressionado com a qualidade do atendi...
6     Fiquei muito satisfeito com a atenção e cuidado.
Name: feedback, dtype: object


In [77]:
def analisar_sentimento(texto):
    analise = TextBlob(texto)
    return analise.sentiment.polarity  # Retorna a polaridade do sentimento


# Traduzir o texto para inglês e aplicar a análise de sentimento
feedback_traduzido = feedback.apply(
    lambda x: GoogleTranslator(source="pt", target="en").translate(x)
)
feedback_polaridade = feedback_traduzido.apply(analisar_sentimento).dropna()


In [ ]:

# Adicionar a polaridade ao DataFrame
cliente["polaridade_feedback"] = feedback_polaridade

In [78]:
# Criando o gráfico de dispersão
fig = px.scatter(
    cliente,
    x=cliente.index,
    y="polaridade_feedback",
    title="Dispersão da Polaridade dos Sentimentos",
    labels={"index": "Índice", "polaridade_feedback": "Polaridade"},
    color="polaridade_feedback",  # Cor de acordo com a polaridade
    color_continuous_scale="Viridis",
    template="plotly_white",
)

# Adicionando linha de referência para sentimentos neutros
fig.add_hline(
    y=0,
    line_color="red",
    annotation_text="Neutro",
    annotation_position="top right",
)

# Exibindo o gráfico
fig.show()